# **Macroeconomia Aberta e DSGE: Fundamentos, Estimação e Aplicações**

João Ricardo Costa Filho \\
https://www.joaocostafilho.com

# **Modelo RBC com duas economias e mercados completos: análise de funções impulso-resposta e estimação bayesiana**

Replicação com base no modelo do artigo [Spurious welfare reversals in international business cycle models](https://www.sciencedirect.com/science/article/abs/pii/S0022199602000478) feita por [Barillas et al. (2019)](https://forum.dynare.org/t/practicing-dynare-2019-updated-version/13389) e adaptada por [João Ricardo Costa Filho](https://www.joaocostafilho.com/).

## Instale na máquina "virtual" o Octave e o Dynare.

In [1]:
!sudo apt-get update &> /dev/null
!sudo apt-get install -y x13as &> /dev/null

In [2]:
!apt-get install octave &> /dev/null

In [3]:
!apt-get install dynare &> /dev/null

## Modelo

### Variáveis

In [4]:
%%writefile variaveis.mod

%----------------------------------------------------------------
% Variáveis endógenas
%----------------------------------------------------------------

var c1      (long_name='Consumo no país 1')
    c2      (long_name='Consumo no país 2')
    k1      (long_name='Estoque de capital físico no país 1')
    k2      (long_name='Estoque de capital físico no país 2')
    a1      (long_name='Produtividade no país 1')
    a2      (long_name='Produtividade no país 2')
;

%----------------------------------------------------------------
% Variáveis exógenas
%----------------------------------------------------------------

varexo e1       (long_name='Choque na produtividade no país 1')
       e2       (long_name='Choque na produtividade no país 2')
;

Writing variaveis.mod


### Parâmetros

In [5]:
%%writefile parametros.mod

%----------------------------------------------------------------
% Parâmetros
%----------------------------------------------------------------

parameters beta  (long_name='Fator de Disconto')
           gamma (long_name='Elasticidade intertemporal de substituição')
           delta (long_name='Taxa de depreciação')
           alpha (long_name='Parcela do capital na função de produção')
             rho (long_name='Persistência da produtividade')
;

%----------------------------------------------------------------
% Calibração
%----------------------------------------------------------------

gamma = 2;
delta =.05;
alpha =.4;
beta  =.98;
rho   =.85;

Writing parametros.mod


### Modelo

In [6]:
%%writefile modelo.mod

%--------------------------------------------------------------------------------------------------------------------------------------
% Modelo
%--------------------------------------------------------------------------------------------------------------------------------------

model;

exp( c1 ) = exp( c2 );

exp(c1)^(-gamma) = beta * exp( c1(+1) )^(-gamma) * ( alpha * exp( a1(+1) ) * exp(k1)^( alpha - 1 ) + 1 - delta );

exp(c2)^(-gamma) = beta * exp( c2(+1) )^(-gamma) * ( alpha * exp( a2(+1) ) * exp(k2)^( alpha - 1 ) + 1 - delta );

exp(c1) + exp(c2) + exp(k1) - exp( k1(-1) ) * ( 1 - delta ) + exp(k2) - exp( k2(-1) ) * ( 1 - delta ) = exp(a1) * exp( k1(-1) )^alpha + exp(a2) * exp( k2(-1) )^alpha;

[name = 'Produtividade no país 1']
a1 = rho * a1(-1) + e1;

[name = 'Produtividade no país 2']
a2 = rho * a2(-1) + e2;


end;

Writing modelo.mod


## Equilíbrio, funções impulso-resposta e estimativa bayesiana

In [7]:
%%writefile IRBC.mod

@#include "variaveis.mod"

@#include "parametros.mod"

@#include "modelo.mod"

%--------------------------------------------------------------------------------------------------------------------------------------
% Equilíbrio
%--------------------------------------------------------------------------------------------------------------------------------------

initval;
k1=2.8;
k2=2.8;
c1=.8;
c2=.8;
a1=0;
a2=0;
e1=0;
e2=0;
end;

resid;
check;

%--------------------------------------------------------------------------------------------------------------------------------------
% Funções impulso-resposta e simulação estocástica
%--------------------------------------------------------------------------------------------------------------------------------------

shocks;
var e1;
stderr .08;
var e2;
stderr .08;
end;

steady;
stoch_simul(periods=1200, drop=200, order=2);

temp.c1 = oo_.endo_simul(1,:);
temp.k2 = oo_.endo_simul(4,:);
save('data.mat', '-struct', 'temp');

%--------------------------------------------------------------------------------------------------------------------------------------
% Estimação Bayesiana
%--------------------------------------------------------------------------------------------------------------------------------------


estimated_params;
rho,        normal_pdf,    .84, .05;
alpha,      normal_pdf,    .38, .03;
stderr e1,  inv_gamma_pdf, .078, inf;
stderr e2,  inv_gamma_pdf, .082, inf;
end;

varobs c1 k2;

estimation(datafile=data, mh_replic=1000, mh_jscale=.5, graph_format=pdf);

Writing IRBC.mod


In [8]:
%%writefile solve.m
dynare IRBC.mod

Writing solve.m


In [9]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 6 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 2).
Processing outputs ...
done
Preprocessing completed.





Residuals of the static equations:

Equation number 1 : 0 : 1
Equation number 2 : -0.00081942 : 2
Equation number 3 : -0.00081942 : 3
Equation number 4 : -0.034162 : 4
Equation number 5 : 0 : Produtividade no país 1
Equation number 6 : 0 : Produtividade no país 2



EIGENVALUES:
         Modulus             Real        Imaginary

       2.001e-15        2.001e-15                0
            0.85             0.85                0
            0.85             0.85                0
          0.9592           0.9592                0
           1.064            1.064                0
       2.088e+13   